# Running and Implementing your First Flows

In [ ]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
import sys
sys.path.append("..")
from utils import compile_and_writefile

#Specify path of your flow modules
FLOW_MODULES_PATH = "./"

## 1. Connect to the CoLink Server

In [ ]:
cl = colink_utils.start_colink_server()

## 2. Reverse Number Atomic

### 2.1 Writing the Reverse Number Flow Class (ACTION REQUIRED)

In [ ]:
%%compile_and_writefile ReverseNumber.py


from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage

class ReverseNumberAtomicFlow(AtomicFlow):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):

        #Get data dictionary from input message
        input_data = input_message.data
        
        #get input number from data dictionary (int)
        input_number = input_data["number"]
        
        #TODO: reverse the input number (e.g. 1234 -> 4321)
        reversed_number = ???
        
        #Create response dictionary
        response = {"reversed_number": reversed_number}
        
        #package ouput message to send back
            #This method packages `response` in a FlowMessage object 
            # containing the necessary metadata to send the message back
            # to the sender of the input message. 
        reply = self.package_output_message(
            input_message=input_message,
            response=response,
        )
        
        #send back reply
        self.send_message(
            reply,
            is_reply=True,
        )



In [ ]:
default_config_reverse_number = \
{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",

    # TODO: Define the target
    "_target_": "ReverseNumberAtomic.ReverseNumberAtomicFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "reversed_number",
}

### 2.2 Serving & Getting and Instance of the Reverse Number Flow

#### 2.2.1 Serving the Reverse Number Flow

In [ ]:
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberAtomicFlow_served", #reference name we give to flow
    default_config=default_config_reverse_number,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    serve_mode="statefull",
)

#### 2.2.2 Getting an instance of the Reverse Number Flow

In [ ]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

# Get an instance of the flow
proxy_reverse_number_flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberAtomicFlow_served", 
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)

### 2.3. Call the Reverse Number Flow via the Proxy

In [ ]:
input_data = {"id": 0, "number": 12345}

# Package your data in a Flow Message

## Option 1: Via the FlowMessage class
# input_message = FlowMessage(
#     data=input_data,
# )

## Option 2 (prefered): Via the package input message method
input_message = proxy_reverse_number_flow.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_number_flow.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()
#To get the response as a FlowMessage object
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")



## 3. Reverse Number Sequential

### 3.1 Writing the Reverse Number Sequential Flow Class (ACTION REQUIRED) 

In [ ]:
%%compile_and_writefile ReverseNumberSequential.py


from aiflows.base_flows import CompositeFlow
from aiflows.messages import FlowMessage
from aiflows.interfaces import KeyInterface
class ReverseNumberSequentialFlow(CompositeFlow):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #~~~~~~~~~~~ Key Transformation solution 1 ~~~~~~~~~~~
        self.input_interface_second_reverse_flow = KeyInterface(
            keys_to_rename= {"reversed_number": "number"},
            keys_to_select= ["number"],
        )
    
        self.ouput_interface_reply = KeyInterface(
            keys_to_rename= {"reversed_number": "output_number"},
            keys_to_select = ["output_number"],
        )
        
        self.get_next_call = {
            "first_reverse_flow": "second_reverse_flow",
            "second_reverse_flow": "reply_to_message",
            "reply_to_message": "first_reverse_flow"
        }
    def set_up_flow_state(self):
        super().set_up_flow_state()
        self.flow_state = {"flow_to_call": "first_reverse_flow"}
        
    def get_next_flow_to_call(self):
        return self.get_next_call[self.flow_state["flow_to_call"]]
        
    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):
        
        #Run here is a bit like a switch statement where we decide which flow to call next.
        # We then call the next flow and pass the input message to it. which we expect to get a reply from 
        # back in the input queue (which will call the run method againg)
        flow_to_call = self.flow_state["flow_to_call"]
        
        #Case where we need to reverse the number for the first time
        if flow_to_call == "first_reverse_flow":
            #Save the initial message to the state
            self.flow_state["initial_message"] = input_message
            
            #Calls the first flow and requests a reply to be sent back to the input queue 
            # (The queue to send back to is specified by self.get_instance_id() --> id of this flow instance
            # of ReverseNumberSequentialFlow)
            self.subflows["first_reverse_flow"].get_reply(
                input_message,
                self.get_instance_id()
            )
        
        #Case where we need to reverse the number for the second time
        elif flow_to_call == "second_reverse_flow":
            
            #Applies a transformation to the input message (renames keys of dictonary so that they match the
            # required format of the second flow)
            message = self.input_interface_second_reverse_flow(input_message)
            
            #TODO: Call the second flow and requests a reply to be sent back to the input queue
            self.subflows["second_reverse_flow"].???
        
        #Case where we need to reply to the initial message (we've already reversed the number twice)
        else:
            message = self.ouput_interface_reply(input_message)
            
            #package ouput message to send back
                #This method packages `response` in a FlowMessage object 
                # containing the necessary metadata to send the message back
                # to the sender of the input message. 
            reply = self.package_output_message(
                input_message = self.flow_state["initial_message"],
                response = message.data
            )
            #send back the reply to initial caller of the flow
            self.send_message(reply, is_reply = True)
            
        self.flow_state["flow_to_call"] = self.get_next_flow_to_call()

In [ ]:
default_config_reverse_number_sequential = \
{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",

    # TODO: Define the target
    "_target_": "ReverseNumberSequential.ReverseNumberSequentialFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "output_number",
    
    "subflows_config": {
        "first_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "A proxy flow that calls reverse number to reverse number AGAIN.",
            "description": "A proxy flow that calls reverse number to reverse number.",
        },
        "second_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "Proxy Second Reverse",
            "description": "A proxy flow that calls reverse number to reverse number AGAIN.",
        },
    }
}


### 3.2 Serving & Getting and Instance of the Reverse Number Flow

#### 3.2.1 Serving the Reverse Number Sequential Flow

In [ ]:
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberSequentialFlow", #reference name we give to flow
    default_config=default_config_reverse_number_sequential,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    serve_mode="statefull",
)

#### 3.2.2 Getting an instance of the Reverse Number Flow

In [ ]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

# Get an instance of the flow
proxy_reverse_number_sequential_flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberSequentialFlow",
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)

### 3.3. Call the Reverse Number Sequential Flow via the Proxy

In [ ]:
input_data = {"id": 0, "number": 12345}

# Package your data in a Flow Message

## Option 1: Via the FlowMessage class
# input_message = FlowMessage(
#     data=input_data,
# )

## Option 2 (prefered): Via the package input message method
input_message = proxy_reverse_number_sequential_flow.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_number_sequential_flow.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()
#To get the response as a FlowMessage object
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")